In [9]:
import wrds
import pandas as pd
import numpy as np

# Loading in S&P500 lookup table

In [10]:
SP500_tickers_table = pd.read_csv('SP500_Tickers.csv')
SP500_permno_list = SP500_tickers_table['PERMNO'].tolist()
display(SP500_tickers_table.head())

,PERMNO,HTICK,HCOMNAM
0,10104,ORCL,ORACLE CORP
1,10107,MSFT,MICROSOFT CORP
2,10138,TROW,T ROWE PRICE GROUP INC
3,10145,HON,HONEYWELL INTERNATIONAL INC
4,10516,ADM,ARCHER DANIELS MIDLAND CO


# Connect to wrds

In [2]:
db = wrds.Connection(username='elliotcky')

Loading library list...
Done


## Get daily OHLC data from wrds from 2010 - 2023 (Not yet adjusted for stock splits)

In [4]:
res = db.raw_sql("""SELECT permno, cusip, date, openprc, askhi, bidlo, prc, vol, cfacpr, cfacshr
                 FROM crsp.dsf 
                 WHERE date >= '2010-01-01'
                 AND permno in {}""".format(tuple(SP500_permno_list)))

In [5]:
display(res.head())

,permno,cusip,date,openprc,askhi,bidlo,prc,vol,cfacpr,cfacshr
0,11600,24906P10,2010-01-04,35.53,35.65,35.2700,35.33,1107552.0,1.0,1.0
1,11403,12738710,2010-01-04,6.01,6.12,5.9500,6.12,3116690.0,1.0,1.0
2,11403,12738710,2010-01-05,6.12,6.12,5.9675,6.06,4002619.0,1.0,1.0
3,11403,12738710,2010-01-06,6.04,6.20,6.0200,6.13,4335079.0,1.0,1.0
4,11403,12738710,2010-01-07,6.10,6.29,6.0900,6.25,4096822.0,1.0,1.0


## Get earnings date from wrds (For calculating eligible starting points for trendlines)

In [6]:
ibes_crsp = pd.read_csv('ibes-to-crsp.csv')
sorted_ibes_crsp = ibes_crsp.sort_values(by=['PERMNO', 'edate'], ascending=[True, False])
sorted_ibes_crsp_no_dup = sorted_ibes_crsp.drop_duplicates(subset=['PERMNO'], keep='first')
sorted_ibes_crsp_no_dup = sorted_ibes_crsp_no_dup.sort_values(by='TICKER', ascending=True)

display(sorted_ibes_crsp_no_dup.head())

,TICKER,PERMNO,NCUSIP,sdate,edate,SCORE
0,003H,14579.0,70432V10,2014-05-15,2022-12-30,1
1,004W,14714.0,04041310,2014-06-19,2022-12-30,1
2,00C6,14939.0,49338L10,2014-11-20,2022-12-30,1
4,00WY,15850.0,57667L10,2020-07-16,2022-12-30,1
5,02J8,18724.0,G0250X10,2019-06-20,2022-12-30,1


### Inner join to only get the earnings date of S&P500 stocks

In [7]:
# Get the intersection of the two dataframes
SP500_cusip_table = SP500_tickers_table.merge(sorted_ibes_crsp_no_dup, how='inner', on='PERMNO')
SP500_cusip_list = SP500_cusip_table['NCUSIP'].tolist()
SP500_cusip_tuple = tuple(SP500_cusip_list)

SP500_EPS = db.raw_sql("select ticker, cusip,  cname, actdats, acttims from tr_ibes.act_epsus where actdats >= '2010-01-01' AND pdicity = 'QTR' AND cusip in {}".format(SP500_cusip_tuple))

In [8]:
display(SP500_EPS.head())

,ticker,cusip,cname,actdats,acttims
0,AFL,00105510,AFLAC INC,2010-02-02,16:58:17
1,AFL,00105510,AFLAC INC,2010-04-27,17:00:01
2,AFL,00105510,AFLAC INC,2010-07-27,16:15:40
3,AFL,00105510,AFLAC INC,2010-10-26,16:25:07
4,AFL,00105510,AFLAC INC,2011-02-01,16:26:52


# Merge S&P500 ticker lookup table with OHLC data

In [11]:
SP500_tickers_table = SP500_tickers_table.rename(columns={'PERMNO': 'permno'})

ohlc_df = res.merge(SP500_tickers_table, how='inner', on='permno')

display(ohlc_df.head())

,permno,cusip,date,openprc,askhi,bidlo,prc,vol,cfacpr,cfacshr,HTICK,HCOMNAM
0,11600,24906P10,2010-01-04,35.53,35.65,35.2700,35.33,1107552.0,1.0,1.0,XRAY,DENTSPLY SIRONA INC
1,11403,12738710,2010-01-04,6.01,6.12,5.9500,6.12,3116690.0,1.0,1.0,CDNS,CADENCE DESIGN SYSTEMS INC
2,11403,12738710,2010-01-05,6.12,6.12,5.9675,6.06,4002619.0,1.0,1.0,CDNS,CADENCE DESIGN SYSTEMS INC
3,11403,12738710,2010-01-06,6.04,6.20,6.0200,6.13,4335079.0,1.0,1.0,CDNS,CADENCE DESIGN SYSTEMS INC
4,11403,12738710,2010-01-07,6.10,6.29,6.0900,6.25,4096822.0,1.0,1.0,CDNS,CADENCE DESIGN SYSTEMS INC


## Adjust for splits

In [12]:
ohlc_df['open'] = ohlc_df['openprc'] / ohlc_df['cfacpr']
ohlc_df['high'] = ohlc_df['askhi'] / ohlc_df['cfacpr']
ohlc_df['low'] = ohlc_df['bidlo'] / ohlc_df['cfacpr']
ohlc_df['close'] = ohlc_df['prc'] / ohlc_df['cfacpr']
ohlc_df['volume'] = ohlc_df['vol'] / ohlc_df['cfacshr']

ohlc_df = ohlc_df.drop(columns=['openprc', 'askhi', 'bidlo', 'prc', 'vol', 'cfacpr', 'cfacshr'])

In [13]:
display(ohlc_df.head())

,permno,cusip,date,HTICK,HCOMNAM,open,high,low,close,volume
0,11600,24906P10,2010-01-04,XRAY,DENTSPLY SIRONA INC,35.53,35.65,35.2700,35.33,1107552.0
1,11403,12738710,2010-01-04,CDNS,CADENCE DESIGN SYSTEMS INC,6.01,6.12,5.9500,6.12,3116690.0
2,11403,12738710,2010-01-05,CDNS,CADENCE DESIGN SYSTEMS INC,6.12,6.12,5.9675,6.06,4002619.0
3,11403,12738710,2010-01-06,CDNS,CADENCE DESIGN SYSTEMS INC,6.04,6.20,6.0200,6.13,4335079.0
4,11403,12738710,2010-01-07,CDNS,CADENCE DESIGN SYSTEMS INC,6.10,6.29,6.0900,6.25,4096822.0


## Determine if earnings occured before hours or after hours

In [19]:
ohlc_df['date'] = pd.to_datetime(ohlc_df['date'])

In [15]:
SP500_EPS['actdats'] = pd.to_datetime(SP500_EPS['actdats'])
SP500_EPS['earnings_time'] = SP500_EPS['acttims'].apply(lambda x: 'AH' if x.hour >= 16 else 'BH')

display(SP500_EPS.head())

,ticker,cusip,cname,actdats,acttims,earnings_time
0,AFL,00105510,AFLAC INC,2010-02-02,16:58:17,AH
1,AFL,00105510,AFLAC INC,2010-04-27,17:00:01,AH
2,AFL,00105510,AFLAC INC,2010-07-27,16:15:40,AH
3,AFL,00105510,AFLAC INC,2010-10-26,16:25:07,AH
4,AFL,00105510,AFLAC INC,2011-02-01,16:26:52,AH


# Finally, merge earnings time data with ohlc data

In [24]:
SP500_EPS = SP500_EPS.rename(columns={'actdats': 'date', 'ticker': 'HTICK'})

ohlc_df = ohlc_df.merge(SP500_EPS, how='left', on=['date', 'HTICK'])
ohlc_df = ohlc_df.drop(columns=['cusip_x', 'cusip_y', 'cname', 'acttims'])

In [25]:
display(ohlc_df.head())

,permno,date,HTICK,HCOMNAM,open,high,low,close,volume,earnings_time
0,11600,2010-01-04,XRAY,DENTSPLY SIRONA INC,35.53,35.65,35.2700,35.33,1107552.0,NaN
1,11403,2010-01-04,CDNS,CADENCE DESIGN SYSTEMS INC,6.01,6.12,5.9500,6.12,3116690.0,NaN
2,11403,2010-01-05,CDNS,CADENCE DESIGN SYSTEMS INC,6.12,6.12,5.9675,6.06,4002619.0,NaN
3,11403,2010-01-06,CDNS,CADENCE DESIGN SYSTEMS INC,6.04,6.20,6.0200,6.13,4335079.0,NaN
4,11403,2010-01-07,CDNS,CADENCE DESIGN SYSTEMS INC,6.10,6.29,6.0900,6.25,4096822.0,NaN
